In [16]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)
server = app.server

launchSiteList = spacex_df['Launch Site'].unique().tolist()
launchSite = []
for x in launchSiteList:
    launchSite.append({'label': x, 'value': x})
launchSite.append({'label': 'All Sites', 'value': 'All Sites'})

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                dcc.Dropdown(id='siteDropdown',options=launchSite,placeholder='Select a Launch Site', searchable = True , value = 'All Sites'),
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(
                                    id='payload_slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    marks = {
                                            0: '0 kg',
                                            1000: '1000 kg',
                                            2000: '2000 kg',
                                            3000: '3000 kg',
                                            4000: '4000 kg',
                                            5000: '5000 kg',
                                            6000: '6000 kg',
                                            7000: '7000 kg',
                                            8000: '8000 kg',
                                            9000: '9000 kg',
                                            10000: '10000 kg'
                                    },
                                    value=[min_payload,max_payload]
                                ),
                                html.Div(dcc.Graph(id='pieChart'), id='plot1', style={'width': '35%', 'display': 'inline-block'}),
                                html.Div(dcc.Graph(id='scatterChart'), id='plot2', style={'width': '65%', 'display': 'inline-block'}),
                                html.Div(),
                                ])

@app.callback(
     Output(component_id='pieChart',component_property='figure'),
     [Input(component_id='siteDropdown',component_property='value')]
)
def update_graph(user_input):
    if (user_input == 'All Sites'):
        df  = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(df, names = 'Launch Site',hole=.3,title = 'Total Success Launches By all sites')
    else:
        df  = spacex_df.loc[spacex_df['Launch Site'] == user_input]
        fig = px.pie(df, names = 'class',hole=.3,title = 'Total Success Launches for site '+user_input)
    
    fig.update_layout(legend=dict(font=dict(size=6)))
    return fig

@app.callback(
     Output(component_id='scatterChart',component_property='figure'),
     [Input(component_id='siteDropdown',component_property='value'),Input(component_id="payload_slider", component_property="value")]
)
def update_scattergraph(user_input,payload_slider):
    if user_input == 'All Sites':
        low, high = payload_slider
        df  = spacex_df
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class",
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    else:
        low, high = payload_slider
        df  = spacex_df.loc[spacex_df['Launch Site'] == user_input]
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class",
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    return fig

if __name__ == '__main__':
    app.run(jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/
